#### A recommender system, or a recommendation system, is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item. They are primarily used in commercial applications. (source - Wikipedia)

#### Mainly three types of recommendation systems in machine learning based on filtering are used to suggest product and services to the consumers.

#### 1. Content Filtering #### 2. Collaborative Filtering #### 3. Hybrid Filtering

   #### 1. Content Filtering:

      In this algorithm, we try finding items look alike. Once we have item look like matrix,

      we can easily recommend alike items to a customer, who has purchased any item from the store.

   #### 2. Collaborative Filtering:

      Here, we try to search for look alike customers and offer products based on what his/her lookalike has chosen.

      This algorithm is very effective but takes a lot of time and resources.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [3]:
tag=pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv')
rating=pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
movies=pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
genome_scores=pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_scores.csv')
link=pd.read_csv('/kaggle/input/movielens-20m-dataset/link.csv')
genome_tag=pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_tags.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [6]:
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [7]:
rating.shape

(20000263, 4)

In [8]:
rating['userId'].value_counts().shape ## unique users

(138493,)

In [10]:
x=rating['userId'].value_counts()>500

In [11]:
y = x[x].index

In [12]:
y.shape

(7441,)

In [13]:
rating=rating[rating['userId'].isin(y)]

In [14]:
rating.shape

(6554416, 4)

In [15]:
movie_details=movies.merge(rating,on='movieId')

In [16]:
movie_details.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0,2001-07-04 07:02:29
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0,2000-11-21 21:00:21
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0,2006-04-03 10:00:08
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0,2005-03-22 22:46:02


In [17]:
movie_details.shape

(6554416, 6)

In [18]:
movie_details.drop(columns=['timestamp'],inplace=True)

In [19]:
movie_details.shape

(6554416, 5)

In [20]:
movie_details.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0


In [21]:
number_rating = movie_details.groupby('title')['rating'].count().reset_index()

In [22]:
number_rating.rename(columns={'rating':'number of rating'},inplace=True)

In [23]:
number_rating.head()

,title,number of rating
0,#chicagoGirl: The Social Network Takes on a Di...,2
1,$ (Dollars) (1971),7
2,$5 a Day (2008),10
3,$9.99 (2008),17
4,$ellebrity (Sellebrity) (2012),1


In [24]:
df=movie_details.merge(number_rating,on='title')

In [25]:
df.shape

(6554416, 6)

In [26]:
df.head()

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,6303
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0,6303
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0,6303
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0,6303
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0,6303


In [27]:
df=df[df['number of rating']>=50] #selecting valuable books by ratings

In [28]:
df.drop_duplicates(['title','userId'],inplace=True)

In [29]:
df.shape

(6396971, 6)

In [30]:
df.head()

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,6303
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0,6303
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0,6303
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0,6303
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0,6303


In [31]:
df.drop(columns=['number of rating'],inplace=True)

In [32]:
df.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4.0


In [33]:
df['rating']=df['rating'].astype(int)

In [34]:
df.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,24,4
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,4
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,58,5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,91,4


In [35]:
movie_pivot=df.pivot_table(columns='userId',index='title',values='rating')

In [36]:
movie_pivot.shape

(9359, 7441)

In [37]:
movie_pivot.fillna(0,inplace=True)

In [38]:
movie_pivot

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
title,,,,,,,,,,,,,,,,,,,,,
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,4.0
'night Mother (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
loudQUIETloud: A Film About the Pixies (2006),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
xXx: State of the Union (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
from scipy.sparse import csr_matrix
movie_sparse=csr_matrix(movie_pivot)

In [40]:


from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors( n_neighbors=7,algorithm='brute',metric='cosine')

In [41]:
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=7)

In [42]:
df.drop(columns=['genres','userId','rating'],inplace=True)

In [43]:
df.drop_duplicates(inplace=True)

In [44]:
df.to_csv('codf.csv',index=False)

In [45]:
distances,suggestions=model.kneighbors(movie_pivot.iloc[540,:].values.reshape(1,-1))

In [46]:
distances

array([[4.44089210e-16, 7.42246328e-01, 7.57713782e-01, 7.70884163e-01,
        7.90774536e-01, 7.93849736e-01, 7.93911073e-01]])

In [47]:
suggestions

array([[ 540, 6149, 8407, 3888,  672, 6732, 6057]])

In [48]:
df1=df.copy()
ti=[]
for i in df1['title']:
    ti.append(i.split(' (')[0])
df1['title']=ti

In [49]:


for i in range(len(suggestions)):
    print(movie_pivot.index[suggestions[i]])


Index(['Article 99 (1992)', 'One Good Cop (1991)', 'Thunderheart (1992)',
       'Hooper (1978)', 'Bad Influence (1990)', 'Q & A (1990)', 'Nuts (1987)'],
      dtype='object', name='title')


In [50]:
def reco(movie_name):
    movie_id=df1[df1['title']=='Toy Story'].drop_duplicates('title')['movieId'].values[0]
    distances,suggestions=model.kneighbors(movie_pivot.iloc[movie_id,:].values.reshape(1,-1))
    
    
    
    for i in range(len(suggestions)):
        return (movie_pivot.index[suggestions[i]])



In [51]:
res=reco("It Conquered the World")

In [52]:
for i in res:
    print(i)

'Til There Was You (1997)
MatchMaker, The (1997)
Evening Star, The (1996)
To Gillian on Her 37th Birthday (1996)
Only You (1994)
Mrs. Winterbourne (1996)
Picture Perfect (1997)
